In [1]:
from tensorflow.keras import preprocessing
import tensorflow as tf
import numpy as np
import boto3
import json
sagemaker = boto3.client('runtime.sagemaker')

with open('macron.txt') as f:
    text = f.read()

tokenizer = preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
max_id = len(tokenizer.word_index)

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [4]:
def next_char(body, temperature=1):
    X_new = preprocess([body])
    result = sagemaker.invoke_endpoint(
        EndpointName='tensorflow-inference-2021-05-26-13-22-36-181',
        ContentType='application/json',
        Body = json.dumps(X_new.numpy().tolist())
    )
    y_proba = result['Body'].read().decode()
    y_proba = json.loads(y_proba)['predictions']
    y_proba = np.array(y_proba)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(body, n_chars=50, temperature=1):
    for _ in range(n_chars):
        body += next_char(body, temperature)
    return body

print('t = .2', complete_text('Mes chers compatri', 150, .2))
print('t = .5', complete_text('Mes chers compatri', 150, .5))
print('t = 1', complete_text('Mes chers compatri', 150, 1))
print('t = 2', complete_text('Mes chers compatri', 150, 2))

t = .2 Mes chers compatriotes, si on va se respecter les règles de la fête, qui ont été partout si on va se respecter les règles de la formation de rester chez eux de nos conc
t = .5 Mes chers compatriotes, je remercie de dire des mesures de s'est du monde. il y aura des métropoles parce que c'est dur. c'est dur. il faut que c'est une règle de pas d
t = 1 Mes chers compatriotes. qui est ce que qu'il y a ça par jour toutes les jeunes, on e comprennent, piendrons clest-à-dire toutes ces prochaines dont on qui sont avoir mo
t = 2 Mes chers compatriotes policiels euh, beauspour partout huit, péenné. encopes, y celony enlenurant tout, cresauceux ça veut rédurs la nappé à,uil est-à-regiquer. on inf
